In [1]:
from IPython.core.interactiveshell import InteractiveShell #python의 대화형 쉘, 인터프리터
InteractiveShell.ast_node_interactivity = "all"#모든 출력값을 연속적으로 출력
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import pyforest
import statistics
import numpy as np
import pandas as pd
import random

from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go

# import statistical libraries
from scipy.stats import norm,skew, boxcox_normmax

pd.options.display.max_rows = 30
pd.options.display.max_columns = 40

In [3]:
abm =  pd.read_csv('../input/airbnb-new-york-city-with-106-features/airbnbmark1.csv')
abm.head(3)
print('abm.shape',abm.shape)
print('abm.size',abm.size)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20200212052319,2020-02-12,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,NaN,Apartment is located on 37th Street between 5t...,"Guests have full access to the kitchen, bathro...",I am a Sound Therapy Practitioner and Kundalin...,"Make yourself at home, respect the space and t...",NaN,NaN,https://a0.muscache.com/im/pictures/f0813a11-4...,NaN,2845,...,94.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20200212052319,2020-02-13,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,NaN,B52 bus for a 10-minute ride to downtown Brook...,"You will have the private, exclusive use of an...","We'll be around, but since you have the top fl...",Smoking - outside please; pets allowed but ple...,NaN,NaN,https://a0.muscache.com/im/pictures/e49999c2-9...,NaN,4869,...,90.0,9.0,9.0,10.0,9.0,10.0,9.0,f,NaN,NaN,f,f,moderate,f,f,1,1,0,0,4.69
2,5099,https://www.airbnb.com/rooms/5099,20200212052319,2020-02-12,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,Read My Full Listing For All Information. New ...,From the apartment is a 10 minute walk to Gran...,I will meet you upon arrival.,I usually check in with guests via text or ema...,• Check-in time is 2PM. • Check-out time is 12...,NaN,NaN,https://a0.muscache.com/im/pictures/24020910/1...,NaN,7322,...,90.0,10.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,moderate,t,t,1,1,0,0,0.59


abm.shape (153254, 106)
abm.size 16244924


## 쓸모있는 column들만 남겨놓기

In [4]:
abm1 = abm[['latitude', 'longitude', 'property_type', 'room_type'
           ,'accommodates', 'bathrooms', 'bedrooms', 'beds'
           ,'bed_type', 'price', 'security_deposit'
            ,'review_scores_rating', 'number_of_reviews', 'amenities']]

In [5]:
abm1.shape
abm1.head(3)

(153254, 14)

,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,review_scores_rating,number_of_reviews,amenities
0,40.75362,-73.98377,Apartment,Entire home/apt,1,1.0,0.0,1.0,Real Bed,$225.00,$350.00,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park..."
1,40.68514,-73.95976,Guest suite,Entire home/apt,3,1.0,1.0,4.0,Real Bed,$89.00,$500.00,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."
2,40.74767,-73.97500,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,$200.00,$300.00,90.0,78,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."


In [6]:
abm1 = abm1.drop_duplicates() #중복된 데이터 행 삭제
print('abm1.shape after dropping duplicate rows: ',abm1.shape)
print('abm1.size:  ',abm1.size)
print('DataTypes wise size: \n', abm1.dtypes.value_counts())

abm1.shape after dropping duplicate rows:  (95201, 14)
abm1.size:   1332814
DataTypes wise size: 
 object     6
float64    6
int64      2
dtype: int64


## 결측값 검사, Data Cleaning

In [7]:
abm1.replace((' '),np.nan,inplace=True) #inplace가 T면 새로운 return 값이 아닌 원본을 아예 수정
abm1.isnull().sum()

latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms                  67
bedrooms                  144
beds                      763
bed_type                    0
price                       0
security_deposit        28150
review_scores_rating    17173
number_of_reviews           0
amenities                   0
dtype: int64

In [8]:
abm1 = abm1.dropna(subset=['bathrooms', 'bedrooms', 'beds', 'review_scores_rating'], how='any', axis=0)
abm1.isnull().sum()

latitude                    0
longitude                   0
property_type               0
room_type                   0
accommodates                0
bathrooms                   0
bedrooms                    0
beds                        0
bed_type                    0
price                       0
security_deposit        19569
review_scores_rating        0
number_of_reviews           0
amenities                   0
dtype: int64

In [9]:
abm1 = abm1.fillna(0) #보증금 nan은 0으로 취급
abm1.isnull().sum()

latitude                0
longitude               0
property_type           0
room_type               0
accommodates            0
bathrooms               0
bedrooms                0
beds                    0
bed_type                0
price                   0
security_deposit        0
review_scores_rating    0
number_of_reviews       0
amenities               0
dtype: int64

In [10]:
print('abm1.shape after dropping nan rows: ',abm1.shape)

abm1.shape after dropping nan rows:  (77698, 14)


In [11]:
def clean_data(df): #$표시 떼어내기. ,구분표 없애기
    for i in ['price','security_deposit']:
        df[i]=df[i].str.replace('$','').str.replace(',', '').astype(float)
    
    df.replace('', np.nan, inplace=True)
    
    return df.head(2)
clean_data(abm1)

,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,review_scores_rating,number_of_reviews,amenities
0,40.75362,-73.98377,Apartment,Entire home/apt,1,1.0,0.0,1.0,Real Bed,225.0,350.0,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park..."
1,40.68514,-73.95976,Guest suite,Entire home/apt,3,1.0,1.0,4.0,Real Bed,89.0,500.0,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."


## 정확도를 위해 리뷰 개수 10개 미만인 호스트는 삭제

In [12]:
indexNames = abm1[abm1['number_of_reviews'] < 10].index
abm1.drop(indexNames , inplace=True)
abm1.shape

(47064, 14)

## Property type을 onehot encoding 하기

In [13]:
# property_type에는 무슨 값이 있을지
abm1['property_type'].value_counts()

Apartment             34519
House                  5255
Townhouse              2452
Loft                   1579
Condominium            1514
Guest suite             720
Hotel                   229
Boutique hotel          181
Serviced apartment      147
Hostel                   93
Guesthouse               86
Bed and breakfast        49
Bungalow                 47
Villa                    40
Other                    38
Tiny house               35
Camper/RV                26
Cottage                  18
Resort                   12
Boat                     10
Earth house               4
Aparthotel                3
Houseboat                 2
Barn                      2
Castle                    2
Cabin                     1
Name: property_type, dtype: int64

In [14]:
Mod_prop_type=abm1['property_type'].value_counts()[5:len(abm1['property_type'].value_counts())].index.tolist()

def change_prop_type(label):
    if label in Mod_prop_type:
        label='Other'
    return label

In [15]:
abm1.loc[:,'property_type'] = abm1.loc[:,'property_type'].apply(change_prop_type)

In [16]:
abm1['property_type'].value_counts() # 5순위 이하 주거형태는 Others로 분류

Apartment      34519
House           5255
Townhouse       2452
Other           1745
Loft            1579
Condominium     1514
Name: property_type, dtype: int64

In [17]:
abm1.loc[abm1['property_type'].str.contains('Apartment'), 'Apartment'] = 1
abm1.loc[abm1['property_type'].str.contains('House'), 'House'] = 1
abm1.loc[abm1['property_type'].str.contains('Other'), 'Other'] = 1
abm1.loc[abm1['property_type'].str.contains('Townhouse'), 'Townhouse'] = 1
abm1.loc[abm1['property_type'].str.contains('Condominium'), 'Condominium'] = 1
abm1.loc[abm1['property_type'].str.contains('Loft'), 'Loft'] = 1
abm1 = abm1.fillna(0)

abm1.isnull().sum()
abm1.head(5)

latitude                0
longitude               0
property_type           0
room_type               0
accommodates            0
bathrooms               0
bedrooms                0
beds                    0
bed_type                0
price                   0
security_deposit        0
review_scores_rating    0
number_of_reviews       0
amenities               0
Apartment               0
House                   0
Other                   0
Townhouse               0
Condominium             0
Loft                    0
dtype: int64

,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,review_scores_rating,number_of_reviews,amenities,Apartment,House,Other,Townhouse,Condominium,Loft
0,40.75362,-73.98377,Apartment,Entire home/apt,1,1.0,0.0,1.0,Real Bed,225.0,350.0,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",1.0,0.0,0.0,0.0,0.0,0.0
1,40.68514,-73.95976,Other,Entire home/apt,3,1.0,1.0,4.0,Real Bed,89.0,500.0,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,0.0,1.0,0.0,0.0,0.0
2,40.74767,-73.97500,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,200.0,300.0,90.0,78,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.0,0.0,0.0,0.0,0.0,0.0
4,40.76489,-73.98493,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,79.0,0.0,84.0,463,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",1.0,0.0,0.0,0.0,0.0,0.0
5,40.80178,-73.96723,Apartment,Private room,1,1.0,1.0,1.0,Real Bed,79.0,0.0,98.0,118,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
abm1.drop('property_type', axis = 1, inplace=True)
abm1.head(3)

,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,review_scores_rating,number_of_reviews,amenities,Apartment,House,Other,Townhouse,Condominium,Loft
0,40.75362,-73.98377,Entire home/apt,1,1.0,0.0,1.0,Real Bed,225.0,350.0,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",1.0,0.0,0.0,0.0,0.0,0.0
1,40.68514,-73.95976,Entire home/apt,3,1.0,1.0,4.0,Real Bed,89.0,500.0,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,0.0,1.0,0.0,0.0,0.0
2,40.74767,-73.97500,Entire home/apt,2,1.0,1.0,1.0,Real Bed,200.0,300.0,90.0,78,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.0,0.0,0.0,0.0,0.0,0.0


## Room_type를 one hot encoding 하기

In [19]:
abm1['room_type'].value_counts()

Entire home/apt    25155
Private room       20646
Shared room          891
Hotel room           372
Name: room_type, dtype: int64

In [20]:
abm1.loc[abm1['room_type'].str.contains('Entire home/apt'), 'Entire home/apt'] = 1
abm1.loc[abm1['room_type'].str.contains('Private room'), 'Private room'] = 1
abm1.loc[abm1['room_type'].str.contains('Shared room'), 'Shared room'] = 1
abm1.loc[abm1['room_type'].str.contains('Hotel room'), 'Hotel room'] = 1
abm1 = abm1.fillna(0)

abm1.drop('room_type', axis = 1, inplace=True)
abm1.isnull().sum()
abm1.head(5)

latitude                0
longitude               0
accommodates            0
bathrooms               0
bedrooms                0
beds                    0
bed_type                0
price                   0
security_deposit        0
review_scores_rating    0
number_of_reviews       0
amenities               0
Apartment               0
House                   0
Other                   0
Townhouse               0
Condominium             0
Loft                    0
Entire home/apt         0
Private room            0
Shared room             0
Hotel room              0
dtype: int64

,latitude,longitude,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,review_scores_rating,number_of_reviews,amenities,Apartment,House,Other,Townhouse,Condominium,Loft,Entire home/apt,Private room,Shared room,Hotel room
0,40.75362,-73.98377,1,1.0,0.0,1.0,Real Bed,225.0,350.0,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,40.68514,-73.95976,3,1.0,1.0,4.0,Real Bed,89.0,500.0,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,40.74767,-73.97500,2,1.0,1.0,1.0,Real Bed,200.0,300.0,90.0,78,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,40.76489,-73.98493,2,1.0,1.0,1.0,Real Bed,79.0,0.0,84.0,463,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,40.80178,-73.96723,1,1.0,1.0,1.0,Real Bed,79.0,0.0,98.0,118,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Bed_type one hot encoding

In [21]:
abm1['bed_type'].value_counts()

Real Bed         46303
Futon              295
Pull-out Sofa      282
Airbed             121
Couch               63
Name: bed_type, dtype: int64

In [22]:
abm1.loc[abm1['bed_type'].str.contains('Real Bed'), 'Real Bed'] = 1
abm1.loc[abm1['bed_type'].str.contains('Futon'), 'Futon'] = 1
abm1.loc[abm1['bed_type'].str.contains('Pull-out Sofa'), 'Pull-out Sofa'] = 1
abm1.loc[abm1['bed_type'].str.contains('Airbed'), 'Airbed'] = 1
abm1.loc[abm1['bed_type'].str.contains('Couch'), 'Couch'] = 1
abm1 = abm1.fillna(0)

abm1.drop('bed_type', axis = 1, inplace=True)
abm1.isnull().sum()
abm1.head(5)

latitude                0
longitude               0
accommodates            0
bathrooms               0
bedrooms                0
beds                    0
price                   0
security_deposit        0
review_scores_rating    0
number_of_reviews       0
amenities               0
Apartment               0
House                   0
Other                   0
Townhouse               0
Condominium             0
Loft                    0
Entire home/apt         0
Private room            0
Shared room             0
Hotel room              0
Real Bed                0
Futon                   0
Pull-out Sofa           0
Airbed                  0
Couch                   0
dtype: int64

,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,review_scores_rating,number_of_reviews,amenities,Apartment,House,Other,Townhouse,Condominium,Loft,Entire home/apt,Private room,Shared room,Hotel room,Real Bed,Futon,Pull-out Sofa,Airbed,Couch
0,40.75362,-73.98377,1,1.0,0.0,1.0,225.0,350.0,94.0,48,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,40.68514,-73.95976,3,1.0,1.0,4.0,89.0,500.0,90.0,307,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,40.74767,-73.97500,2,1.0,1.0,1.0,200.0,300.0,90.0,78,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,40.76489,-73.98493,2,1.0,1.0,1.0,79.0,0.0,84.0,463,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,40.80178,-73.96723,1,1.0,1.0,1.0,79.0,0.0,98.0,118,"{Internet,Wifi,""Air conditioning"",""Paid parkin...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
